# Install Requirements

In [2]:
!pip install telethon PyDrive2 > /dev/null
print("✔️ Packages installed")

✔️ Packages installed


# Mount Google Drive

In [1]:
from google.colab import drive
import os

if not os.path.exists('/content/drive/MyDrive'):
    drive.mount('/content/drive')
    !mkdir -p "/content/drive/MyDrive/Telegram Downloads"
    print("✅ Drive mounted at /content/drive/MyDrive/Telegram Downloads")

Mounted at /content/drive
✅ Drive mounted at /content/drive/MyDrive/Telegram Downloads


#Speed Test

In [1]:
import requests
import time

def test_speed():
    # download a small test file
    test_url = "http://speedtest.tele2.net/10MB.zip"

    print("Testing Download Speed...")
    start = time.time()
    r = requests.get(test_url, stream=True)
    size = 0
    for chunk in r.iter_content(chunk_size=1024*1024):  # 1MB chunks
        size += len(chunk)
        if size >= 10*1024*1024:
            break
    end = time.time()

    speed = (10 / (end - start))  # MB/s
    print(f"Current Speed: {speed:.2f} MB/s")

    if speed > 5:
      time.sleep(1)
      print("Lets F**king Do it 😁")
    else:
      time.sleep(2)
      print("Oh, Perhaps You Might Try Later! 😞")

test_speed()

Testing Download Speed...
Current Speed: 7.42 MB/s
Lets F**king Do it 😁


# Login with Telegram Credentials

In [ ]:
from telethon import TelegramClient
import asyncio
import os

# enter Telegram Credentials
API_ID = 'your_api_id_here'
API_HASH = 'your_api_hash_here'
PHONE = '+1xxxxxxxxx'

# save session to Google Drive
SESSION_FILE = 'telegram_session.session'

async def login():
    client = TelegramClient(SESSION_FILE, API_ID, API_HASH)
    await client.start(phone=PHONE)
    me = await client.get_me()
    print(f"✔️ Logged in as: {me.username or me.first_name}")
    return client

print("🔄 Logging into Telegram...")
client = await login()
print("✔️ Logged in! Session Saved in Colab")

🔄 Logging into Telegram...
Please enter the code you received: 58484
Please enter your password: ··········
Signed in successfully as MAHDI MASOUMI; remember to not break the ToS or you will risk an account ban!
✔️ Logged in as: MAPTZUY575S461LEH0R1GV87QXB239KC
✔️ Logged in! Session Saved in Colab


# Get the Channel ID

In [4]:
channel_username = "iranianfilmarch"  # Change for different channel
message_id = 881  # Change for different file

async def setup_channel():
    entity = await client.get_entity(channel_username)
    channel_id = int(f"-100{entity.id}")

    global CHANNEL_ID, CHANNEL_TITLE, CURRENT_MESSAGE_ID
    CHANNEL_ID = channel_id
    CHANNEL_TITLE = entity.title
    CURRENT_MESSAGE_ID = message_id

    msg = await client.get_messages(entity, ids=message_id)
    file_name = msg.file.name if msg and msg.media else "No file"

    print(f"✅ Channel: {CHANNEL_TITLE}")
    print(f"✅ ID: {CHANNEL_ID}")
    print(f"☑️ Message: {CURRENT_MESSAGE_ID}")
    print(f"📄 File: {file_name}")
    print("🎯 Ready to Download!")

await setup_channel()

NameError: name 'client' is not defined

# Download The File With Colab

In [33]:
import time
from tqdm.notebook import tqdm
import os

custom_filename = ""  # Leave empty to use formatted name

async def download_only():
    try:
        # Get the channel entity
        channel = await client.get_entity(CHANNEL_ID)
        print(f"✅ Connected to: {channel.title}")

        # Get the specific message
        message = await client.get_messages(channel, ids= CURRENT_MESSAGE_ID)

        if message and message.media:
            # Get original filename
            original_name = message.file.name or f'message_{CURRENT_MESSAGE_ID}'
            # Create formatted name
            formatted_name = original_name.replace('_', '.')
            # Use custom name if provided otherwise use formatted name
            file_name = custom_filename if custom_filename else formatted_name

            file_size = message.file.size
            size_mb = file_size / (1024 * 1024)

            print(f"📄 Original File: {original_name}")
            print(f"📊 Size: {size_mb:.2f} MB")
            print("⬇️ Downloading From Telegram...")

            # Create progress bar
            progress = tqdm(
                total=file_size,
                unit='B',
                unit_scale=True,
                desc=file_name[:30] + "..." if len(file_name) > 30 else file_name,
                bar_format='{l_bar}{bar:40}{r_bar}{bar:-10b}'
            )

            # Download with callback
            def progress_callback(current, total):
                progress.update(current - progress.n)

            path = await message.download_media(
                progress_callback= progress_callback
                # part_size_kb= 1024  # 1MB chunks for stability
            )
            progress.close()
            print(f"✅ DOWNLOAD COMPLETE!")

            global DOWNLOADED_FILE_PATH, DOWNLOADED_FILE_NAME
            DOWNLOADED_FILE_PATH = path
            DOWNLOADED_FILE_NAME = file_name

            print(f"\n⚠️ File Is In Colab Temp Storage")
            print(f"📁 Will Upload As: {DOWNLOADED_FILE_NAME}")
            print(f"⏩ Now Run the Upload Cell")

        else:
            print("❌ No Media Found in This Message!")

    except Exception as e:
        print(f"❌ Error: {e}")

await download_only()

✅ Connected to: Iranian Film Archive
📄 Original file: Marg.Yazdgerd.AKA.Death.of.Yazdgerd.1982.DVDRip.x264-Arya.mkv
📊 Size: 1635.92 MB
⬇️ Downloading from Telegram...


Marg.Yazdgerd.AKA.Death.of.Yaz...:   0%|                                        | 0.00/1.72G [00:00<?, ?B/s]

✅ DOWNLOAD COMPLETE!

⚠️ File Is In Colab Temp Storage
📁 Will upload as: Marg.Yazdgerd.AKA.Death.of.Yazdgerd.1982.DVDRip.x264-Arya.mkv
⏩ Now run the Upload Cell


# Upload to Google Dirve

In [34]:
import os
from tqdm.notebook import tqdm

# Use the file from download cell
try:
    source_file = DOWNLOADED_FILE_PATH
    file_name = DOWNLOADED_FILE_NAME
except NameError:
    print("❌ ERROR: No File Found! Run the DOWNLOAD Cell First.")
    print("📁 Files In Temp Directory:")
    !ls -lh | grep -v total
    source_file = ""  # Will cause exit
    file_name = ""

# Check if source file exists
if source_file and os.path.exists(source_file):
    file_size = os.path.getsize(source_file)
    size_mb = file_size / (1024 * 1024)

    print(f"\n📄 Source: {os.path.basename(source_file)}")
    print(f"📊 Size: {size_mb:.2f} MB")
    print(f"📁 Destination Name: {file_name}")

    # Create destination foler
    drive_folder = "/content/drive/MyDrive/Telegram Downloads"
    !mkdir -p "{drive_folder}"
    print(f"☑️ Destination Folder Ready")

    # Upload with verification
    dest_path = f"{drive_folder}/{file_name}"

    # Handle duplicate names
    counter = 1
    while os.path.exists(dest_path):
        name, ext = os.path.splitext(file_name)
        dest_path = f"{drive_folder}/{name}_{counter}{ext}"
        counter += 1

    print(f"\n☁️ Uploading to Google Drive...")
    print(f"📍 Destination: {dest_path}")
    !cp "{source_file}" "{dest_path}"

    # Verify the file was copied
    if os.path.exists(dest_path):
        uploaded_size = os.path.getsize(dest_path)
        uploaded_mb = uploaded_size / (1024 * 1024)

        print(f"\n✅ UPLOAD SUCCESSFUL!")

        # VERIFY STORAGE
        print(f"💾 Current Drive Storage Usage:" )
        !du -sh "/content/drive/MyDrive" | cut -f1

        print(f"\n✨ DONE! File is in Google Drive")
        print(f"📍 Folder: Telegram Downloads")
        print(f"📁 Filename: {os.path.basename(dest_path)}")

    else:
        print(f"\n❌ Upload Failed - File NOT Found at Destination!")

else:
    if source_file:
        print(f"\n❌ Error: File '{source_file}' NOT Found in Colab Temp!")
    print("\n📁 Files Currently in Colab Temp:")
    !ls -lh | grep -v total


📄 Source: Marg.Yazdgerd.AKA.Death.of.Yazdgerd.1982.DVDRip.x264-Arya.mkv
📊 Size: 1635.92 MB
📁 Destination Name: Marg.Yazdgerd.AKA.Death.of.Yazdgerd.1982.DVDRip.x264-Arya.mkv
☑️ Destination Folder Ready

☁️ Uploading to Google Drive...
📍 Destination: /content/drive/MyDrive/Telegram Downloads/Marg.Yazdgerd.AKA.Death.of.Yazdgerd.1982.DVDRip.x264-Arya.mkv

✅ UPLOAD SUCCESSFUL!
💾 Current Drive Storage Usage:
15G

✨ DONE! File is in Google Drive
📍 Folder: Telegram Downloads
📁 Filename: Marg.Yazdgerd.AKA.Death.of.Yazdgerd.1982.DVDRip.x264-Arya.mkv


# **Cache Clean Up**

In [35]:
import os
import glob

print("🧹 Cleaning up Colab Temp Files...")

# ONLY DELETE MEDIA FILES
deleted_count = 0
for pattern in ['*.mkv', '*.rar', '*.mp4', '*.zip', '*.part', '*.mp3']:
    files = glob.glob(pattern)
    for f in files:
        try:
            os.remove(f)
            print(f"  Deleted: {f}")
            deleted_count += 1
        except:
            pass

print(f"\n✅ Deleted {deleted_count} Downloaded Files")

# SAFELY CHECK DRIVE STILL MOUNTED
if os.path.exists('/content/drive/MyDrive'):
    print("✅ Drive Mount Intact - Session Files Safe")
else:
    print("⚠️ Drive Disconnected - Remount Needed")

print("\n✨ Clean-Up Completed")

🧹 Cleaning up Colab Temp Files...
  Deleted: Marg.Yazdgerd.AKA.Death.of.Yazdgerd.1982.DVDRip.x264-Arya.mkv
  Deleted: 1979_Ballad_of_Tara.zip

✅ Deleted 2 Downloaded Files
✅ Drive Mount Intact - Session Files Safe

✨ Clean-Up Completed
